In [1]:
import logging
from oemof.tools import logger
logger.define_logging()

logging.info('The program started')

16:35:48-INFO-Path for logging: /home/local/RL-INSTITUT/caroline.moeller/.oemof/log_files
16:35:48-INFO-Used oemof version: 0.1.4
16:35:48-INFO-The program started


In [2]:
# INSTANTIATE ENERGY SYSTEM

import pandas as pd
from oemof import solph

date_time_index = pd.date_range('1/1/2017', periods=8760,
                               freq='H')

energysystem = solph.EnergySystem(timeindex=date_time_index)

In [3]:
print(energysystem)

In [5]:
import pandas as pd

# Import wind, PV and demand data
data = pd.read_csv('data/example_data.csv', sep=',')

In [6]:
print(data)

      timestep      demand_el        pv      wind
0            1  209643.341577  0.000000  0.107020
1            2  207497.200771  0.000000  0.074873
2            3  200108.014899  0.000000  0.055654
3            4  191892.680200  0.000000  0.065059
4            5  185717.333107  0.000000  0.085573
5            6  180672.748373  0.000000  0.096229
6            7  172683.566147  0.000000  0.117940
7            8  170048.197544  0.000000  0.159530
8            9  171132.806339  0.000000  0.237170
9           10  179532.755300  0.062188  0.285150
10          11  189155.773753  0.154260  0.357240
11          12  201026.470857  0.160160  0.463540
12          13  208466.425651  0.103600  0.569320
13          14  207718.737886  0.084862  0.689990
14          15  205443.367096  0.052101  0.795170
15          16  206255.669853  0.014673  0.868930
16          17  217240.218495  0.000000  0.922780
17          18  232798.585499  0.000000  0.958950
18          19  237321.634940  0.000000  0.979630


In [7]:
# Define cost parameter

from oemof.tools import economics

price_gas = 0.04

epc_wind = economics.annuity(capex=1000, n=20, wacc=0.05)  # equivalent periodical costs

epc_pv = economics.annuity(capex=1000, n=20, wacc=0.05)

epc_storage = economics.annuity(capex=1000, n=10, wacc=0.05)

In [8]:
# POPULATE ENERGY SYSTEM

from oemof import solph

# Create busses
bgas = solph.Bus(label='natural_gas')

bel = solph.Bus(label='electricity')

In [9]:
# Create sources

solph.Source(label='gas',
             outputs={bgas: solph.Flow(variable_costs=price_gas)})

solph.Source(label='wind',
             outputs={bel: solph.Flow(
                 actual_value=data['wind'],
                 fixed=True,
                 fixed_costs=20,
                 investment=solph.Investment(ep_costs=epc_wind))})

solph.Source(label='pv',
             outputs={bel: solph.Flow(
                 actual_value=data['pv'],
                 fixed=True,
                 fixed_costs=20,
                 investment=solph.Investment(ep_costs=epc_pv))})

In [10]:
# Create sinks

solph.Sink(label='demand',
           inputs={bel: solph.Flow(
               actual_value=data['demand_el'],
               fixed=True,
               nominal_value=1)})

solph.Sink(label='excess_bel',
           inputs={bel: solph.Flow()})

In [11]:
# Create transformer

solph.LinearTransformer(label='pp_gas',
                        inputs={bgas: solph.Flow()},
                        outputs={bel: solph.Flow(
                            nominal_value=10e10,
                            variable_costs=0)},
                        conversion_factors={bel: 0.58})

In [12]:
# Create storage

solph.Storage(label='storage',
              inputs={bel: solph.Flow(variable_costs=1)},
              outputs={bel: solph.Flow()},
              capacity_loss=0,
              initial_capacity=0,
              nominal_input_capacity_ratio=1/6,
              nominal_output_capacity_ratio=1/6,
              inflow_conversion_factor=1,
              outflow_conversion_factor=0.8,
              fixed_costs=35,
              investment=solph.Investment(ep_costs=epc_storage))

In [ ]:
# COMPUTE RESULTS

om = solph.OperationalModel(energysystem)

om.solve(solver='cbc', solve_kwargs={'tee': True})

Welcome to the CBC MILP Solver 
Version: 2.8.12 
Build Date: Sep  8 2014 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpyj4qzisg.pyomo.lp -import -stat=1 -solve -solu /tmp/tmpyj4qzisg.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Current default (if $ as parameter) for import is /tmp/tmpyj4qzisg.pyomo.lp


In [ ]:
# filename = os.path.join(
#             helpers.extend_basic_path('lp_files'), 'storage_invest.lp')
# logging.info('Store lp-file in {0}.'.format(filename))
# om.write(filename, io_options={'symbolic_solver_labels': True})

In [ ]:
# PROCESS RESULTS

from oemof import outputlib

storage = energysystem.groups['storage']
wind_inst = energysystem.groups['wind']
pv_inst = energysystem.groups['pv']
myresults = outputlib.DataFramePlot(energy_system=energysystem)

pp_gas = myresults.slice_by(obj_label='pp_gas', type='to_bus',
                            date_from='2017-01-01 00:00:00',
                            date_to='2017-12-31 23:00:00')

demand = myresults.slice_by(obj_label='demand',
                            date_from='2017-01-01 00:00:00',
                            date_to='2017-12-31 23:00:00')

import pprint as pp
pp.pprint({
    'wind_inst_MW': energysystem.results[wind_inst][bel].invest/1000,
    'pv_inst_MW': energysystem.results[pv_inst][bel].invest/1000,
    'storage_cap_GWh': energysystem.results[storage][storage].invest/1e6,
    'res_share': 1 - pp_gas.sum()/demand.sum(),
    'objective': energysystem.results.objective
    })

In [ ]:
import matplotlib.pyplot as plt

cdict = {'wind': '#5b5bae',
         'pv': '#ffde32',
         'storage': '#42c77a',
         'pp_gas': '#636f6b',
         'demand': '#ce4aff',
         'excess_bel': '#555555'}

myplot = outputlib.DataFramePlot(energy_system=energysystem)
# Plotting the balance around the electricity plot for one week using a
# combined stacked plot
fig = plt.figure(figsize=(24, 14))
plt.rc('legend', **{'fontsize': 19})
plt.rcParams.update({'font.size': 19})
plt.style.use('grayscale')

handles, labels = myplot.io_plot(
    bus_label='electricity', cdict=cdict,
    barorder=['pv', 'wind', 'pp_gas', 'storage'],
    lineorder=['demand', 'storage', 'excess_bel'],
    line_kwa={'linewidth': 4},
    ax=fig.add_subplot(1, 1, 1),
    date_from="2017-06-01 00:00:00",
    date_to="2017-06-08 00:00:00",
    )
myplot.ax.set_ylabel('Power in kW')
myplot.ax.set_xlabel('Date')
myplot.ax.set_title("Electricity bus")
myplot.set_datetime_ticks(tick_distance=24, date_format='%d-%m-%Y')
myplot.outside_legend(handles=handles, labels=labels)

plt.show()